In [10]:
# Requirements:
import numpy as np, glob
from scipy import misc
import scipy
from astropy.io import fits
from PIL import Image
from astropy.stats import sigma_clip

In [2]:
# setup
from ginga.web.pgw import ipg
# Set this to True if you have a non-buggy python OpenCv bindings--it greatly speeds up some operations
use_opencv = False

server = ipg.make_server(host='localhost', port=9914, use_opencv=use_opencv)

In [3]:
# Start viewer server
# IMPORTANT: if running in an IPython/Jupyter notebook, use the no_ioloop=True option
server.start(no_ioloop=True)

# Get a viewer
# This will get a handle to the viewer v1 = server.get_viewer('v1')
v1 = server.get_viewer('v1')

# where is my viewer
v1.url

'http://localhost:9914/app?id=v1'

In [4]:
# set a color map on the viewer 
v1.set_color_map('gray')

# Set color distribution algorithm
# choices: linear, log, power, sqrt, squared, asinh, sinh, histeq, 
v1.set_color_algorithm('linear')
# Set cut level algorithm to use
v1.set_autocut_params('zscale', contrast=0.25)
# Auto cut levels on the image
v1.auto_levels()
v1.set_window_size(650, 210)
#v1.rotate(90)
#v1.set_window_size(210, 650)

In [20]:
cutoff_percent = 1.15
def cutoff_forw(max_vals):
    cutoff_point = 0
    prev_val = 0
    curr_val = 0
    for idx in range(1,len(max_vals)):
        prev_val = max_vals[idx-1]
        curr_val = max_vals[idx]
    
        if curr_val > (cutoff_percent * prev_val):
            cutoff_point = idx
            break

    return cutoff_point

def cutoff_back(max_vals):
    max_vals.reverse()
    cutoff_point = 0
    prev_val = 0
    curr_val = 0
    for idx in range(1,len(max_vals)):
        prev_val = max_vals[idx-1]
        curr_val = max_vals[idx]
    
        if curr_val > (cutoff_percent * prev_val):
            cutoff_point = idx
            break

    return cutoff_point

In [21]:
data_locations = [ \
    "images/arc/*.fits", \
    "images/bias/*.fits", \
    "images/science/*.fits", \
    "images/standard/*.fits", \
    "images/flat/*.fits"]

In [22]:
for index, location in enumerate(data_locations):
    images = glob.glob(location)
    for image_file in images:
        image_title = image_file.split("/")
        names = image_title[-1].split(".")
        cls = image_title[1]
        filename = names[0]

        fits_f = fits.open(image_file, 'readonly')
        hdu = fits_f[0]
        image = hdu.data
        shape = image.shape

        if shape[0] > shape[1]:
            image = scipy.ndimage.interpolation.rotate(image,angle=90.0)
            filtered_data = sigma_clip(image, sigma=3, axis=0).tolist()

            max_vals = []
            for idx, val in enumerate(filtered_data):
                max_vals.append(max(val))
            cutoff_f = cutoff_forw(max_vals)
            cutoff_b = cutoff_back(max_vals)
            first  = cutoff_f
            second = shape[0] - cutoff_b
            
            image = image[first:second,:]

            v1.load_data(image)
        else:
            v1.load_data(image)
            
        filename = "./images/" + cls + "/linear_" + filename + ".png"
        v1.save_rgb_image_as_file(filename, format='png')
        print(filename)


./images/arc/linear_b1000.png
./images/arc/linear_b1001.png
./images/arc/linear_b1002.png
./images/arc/linear_b1003.png
./images/arc/linear_b1004.png
./images/arc/linear_b1005.png
./images/arc/linear_b122.png
./images/arc/linear_b123.png
./images/arc/linear_b144.png
./images/arc/linear_b145.png
./images/arc/linear_r1000.png
./images/arc/linear_r1001.png
./images/arc/linear_r1002.png
./images/arc/linear_r1003.png
./images/arc/linear_r1004.png
./images/arc/linear_r1005.png
./images/arc/linear_r138.png
./images/arc/linear_r139.png
./images/arc/linear_r140.png
./images/arc/linear_r141.png
./images/arc/linear_r162.png
./images/arc/linear_r163.png
./images/bias/linear_apr6_2016_b9.png
./images/bias/linear_b157.png
./images/bias/linear_b158.png
./images/bias/linear_b159.png
./images/bias/linear_b160.png
./images/bias/linear_b161.png
./images/bias/linear_b162.png
./images/bias/linear_b163.png
./images/bias/linear_b164.png
./images/bias/linear_b165.png
./images/bias/linear_b166.png
./images/bia